In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import re
import json
import pickle
import textstat
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from pprint import pprint
import plotly.express as px
from scipy.stats import spearmanr
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_splitter import SentenceSplitter
from collections import Counter

In [20]:
df = pd.read_csv("gsm8k_unified_llm_response_data_v4.csv")
# df = pd.read_csv("mmlu_unified_llm_response_data_v4.csv")
df

,question,gold_answer,split,sr_falcon-7b,sr_gemma-7b,sr_llama2-7b-lm,sr_metamath-7b,sr_mistral-7b-lm,sr_olmo-7b,sr_opt-6.7b,...,maj_gemma-7b-it,maj_llama2-13b-chat,maj_mistral-7b-inst,auto_answers_gemma-7b-it,auto_answers_llama2-13b-chat,auto_answers_mistral-7b-inst,rl_gemma-7b,rl_mistral-7b-lm,rl_llama2-13b-chat,rl_gemma-7b-it
0,Natalia sold clips to 48 of her friends in Apr...,72.0,train,0.8,0.9,0.8,1.0,1.0,0.8,0.1,...,1.0,1.0,1.0,"[72.0, 24.0, 72.0, 72.0, 72.0, 'INVALID', 72.0...","[72.0, 72.0, 72.0, 72.0, 720.0, 72.0, 24.0, 72...","[96.0, 24.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....",0.142857,0.142857,0.142857,0.142857
1,Weng earns $12 an hour for babysitting. Yester...,10.0,train,0.0,0.8,0.1,1.0,0.5,0.1,0.0,...,0.0,0.0,1.0,"[6.0, 6.0, 6.0, 6.0, 6.0, 'INVALID', 'INVALID'...","[60.0, 9.96, 60.0, 60.0, 60.0, 60.0, 'INVALID'...","[9.99992, 10.0, 12.0, 10.4, 10.0, 9.996, 12.0,...",0.428571,0.428571,0.047619,0.047619
2,Betty is saving money for a new wallet which c...,5.0,train,0.1,0.5,0.1,1.0,0.2,0.0,0.1,...,0.0,1.0,1.0,"[-5.0, 30.0, 'INVALID', 45.0, 30.0, 35.0, 35.0...","[45.0, 45.0, 45.0, 5.0, 55.0, 5.0, 5.0, 55.0, ...","[5.0, 75.0, 5.0, 105.0, 55.0, 5.0, 92.5, 25.0,...",0.428571,0.047619,0.428571,0.047619
3,"Julie is reading a 120-page book. Yesterday, s...",42.0,train,0.0,0.6,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,"[72.0, 84.0, 72.0, 42.0, 48.0, 84.0, 24.0, 42....","[60.0, 64.0, 60.0, 44.0, 96.0, 84.0, 42.0, 96....","[42.0, 72.0, 42.0, 54.0, 12.0, 60.0, 24.0, 'IN...",0.428571,0.047619,0.047619,0.428571
4,James writes a 3-page letter to 2 different fr...,624.0,train,0.0,0.1,0.1,1.0,0.1,0.0,0.0,...,0.0,0.0,0.0,"[208.0, 3.0, 104.0, 104.0, 'INVALID', 156.0, 1...","[312.0, 312.0, 312.0, 312.0, 624.0, 312.0, 208...","[624.0, 312.0, 1296.0, 312.0, 312.0, 312.0, 62...",0.428571,0.428571,0.428571,0.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8787,John had a son James when he was 19. James is...,8.0,test,0.0,0.0,0.0,0.3,0.0,0.0,0.0,...,0.0,0.0,0.0,"['INVALID', 17.0, 14.0, 8.0, 'INVALID', 10.0, ...","[14.0, 11.0, 16.0, 35.0, 11.0, 8.0, 16.0, 15.0...","['INVALID', 6.0, 'INVALID', 38.0, 'INVALID', 3...",0.428571,0.428571,0.428571,0.428571
8788,There are some oranges in a basket. Ana spends...,5.0,test,0.0,0.6,0.0,0.9,0.6,0.0,0.0,...,0.0,1.0,1.0,"['INVALID', 'INVALID', 5.0, 9.0, 2.0, 'INVALID...","[5.0, 9.0, 'INVALID', 5.0, 60.0, 0.0, 30.0, 1....","[5.0, 5.0, 'INVALID', 5.0, 5.0, 'INVALID', 5.0...",0.238095,0.238095,0.238095,0.000000
8789,Mark's car breaks down and he needs to get a n...,230.0,test,0.0,0.4,0.0,1.0,0.3,0.0,0.0,...,1.0,0.0,0.0,"[230.0, 230.0, 230.0, 950.0, 230.0, 950.0, 950...","[230.0, 470.0, 230.0, 470.0, 470.0, 470.0, 960...","[230.0, 470.0, 80.0, 230.0, 230.0, 470.0, 470....",0.428571,0.047619,0.047619,0.428571
8790,"Farmer Brown has 20 animals on his farm, all e...",5.0,test,0.0,0.1,0.0,0.3,0.0,0.1,0.2,...,0.0,0.0,1.0,"[0.0, 'INVALID', 3.0, 'INVALID', 'INVALID', 10...","[30.0, 33.0, 23.0, 15.0, 35.0, 35.0, 30.0, 20....","[15.0, 10.0, 14.0, 5.0, 5.0, 5.0, 5.0, 5.0, 17...",0.428571,0.428571,0.428571,0.428571


In [21]:
def get_majority_vote(lst):
    return Counter(lst).most_common(n=1)[0][0]

for c in df.columns:
    if c.startswith("auto_answers"):
        df[c] = [eval(v) for v in df[c]]
        llm = c.split('_')[-1]
        print(llm)
        df[f"maj_{llm}"] = [float(get_majority_vote(v)==g) for v,g in zip(df[c],df["gold_answer"])]
        # df[f"maj_{llm}_CHAT"] = [float(get_majority_vote(v)==g) for v,g in zip(df[c],df["gold_answer"])]

falcon-7b
gemma-7b
llama2-7b-lm
metamath-7b
mistral-7b-lm
olmo-7b
opt-6.7b
gemma-7b-it
llama2-13b-chat
mistral-7b-inst


In [22]:
llms = [
"llama2-7b-lm",
"gemma-7b",
"mistral-7b-lm",
"metamath-7b",
"gemma-7b-it",
"llama2-13b-chat",
"mistral-7b-inst",
]

# llms = ['_'.join(c.strip().split('_')[-1:]) for c in df.columns if c.startswith('sr_')]
# # exclude_list = ["gemma-7b_LM","mistral-7b-lm_LM"]
# # llms = [llm for llm in llms if llm not in exclude_list]

llm_model_names = [
    "google/gemma-7b-it",
    "meta-llama/Llama-2-13b-chat-hf",
    "mistralai/Mistral-7B-Instruct-v0.1",
]

tokenizers = [
    AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
    ) for model_name in llm_model_names
]

qset = [s.strip() for s in df["question"]]
# qset = [s.strip() for s in df["question"]]
# qset = gsols

In [23]:
llms

['llama2-7b-lm',
 'gemma-7b',
 'mistral-7b-lm',
 'metamath-7b',
 'gemma-7b-it',
 'llama2-13b-chat',
 'mistral-7b-inst']

In [27]:
df

,question,gold_answer,split,sr_falcon-7b,sr_gemma-7b,sr_llama2-7b-lm,sr_metamath-7b,sr_mistral-7b-lm,sr_olmo-7b,sr_opt-6.7b,...,maj_gemma-7b-it,maj_llama2-13b-chat,maj_mistral-7b-inst,auto_answers_gemma-7b-it,auto_answers_llama2-13b-chat,auto_answers_mistral-7b-inst,rl_gemma-7b,rl_mistral-7b-lm,rl_llama2-13b-chat,rl_gemma-7b-it
0,Natalia sold clips to 48 of her friends in Apr...,72.0,train,0.8,0.9,0.8,1.0,1.0,0.8,0.1,...,1.0,1.0,1.0,"[72.0, 24.0, 72.0, 72.0, 72.0, INVALID, 72.0, ...","[72.0, 72.0, 72.0, 72.0, 720.0, 72.0, 24.0, 72...","[96.0, 24.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....",0.142857,0.142857,0.142857,0.142857
1,Weng earns $12 an hour for babysitting. Yester...,10.0,train,0.0,0.8,0.1,1.0,0.5,0.1,0.0,...,0.0,0.0,1.0,"[6.0, 6.0, 6.0, 6.0, 6.0, INVALID, INVALID, 9....","[60.0, 9.96, 60.0, 60.0, 60.0, 60.0, INVALID, ...","[9.99992, 10.0, 12.0, 10.4, 10.0, 9.996, 12.0,...",0.428571,0.428571,0.047619,0.047619
2,Betty is saving money for a new wallet which c...,5.0,train,0.1,0.5,0.1,1.0,0.2,0.0,0.1,...,0.0,1.0,1.0,"[-5.0, 30.0, INVALID, 45.0, 30.0, 35.0, 35.0, ...","[45.0, 45.0, 45.0, 5.0, 55.0, 5.0, 5.0, 55.0, ...","[5.0, 75.0, 5.0, 105.0, 55.0, 5.0, 92.5, 25.0,...",0.428571,0.047619,0.428571,0.047619
3,"Julie is reading a 120-page book. Yesterday, s...",42.0,train,0.0,0.6,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,"[72.0, 84.0, 72.0, 42.0, 48.0, 84.0, 24.0, 42....","[60.0, 64.0, 60.0, 44.0, 96.0, 84.0, 42.0, 96....","[42.0, 72.0, 42.0, 54.0, 12.0, 60.0, 24.0, INV...",0.428571,0.047619,0.047619,0.428571
4,James writes a 3-page letter to 2 different fr...,624.0,train,0.0,0.1,0.1,1.0,0.1,0.0,0.0,...,0.0,0.0,0.0,"[208.0, 3.0, 104.0, 104.0, INVALID, 156.0, 104...","[312.0, 312.0, 312.0, 312.0, 624.0, 312.0, 208...","[624.0, 312.0, 1296.0, 312.0, 312.0, 312.0, 62...",0.428571,0.428571,0.428571,0.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8787,John had a son James when he was 19. James is...,8.0,test,0.0,0.0,0.0,0.3,0.0,0.0,0.0,...,0.0,0.0,0.0,"[INVALID, 17.0, 14.0, 8.0, INVALID, 10.0, 16.0...","[14.0, 11.0, 16.0, 35.0, 11.0, 8.0, 16.0, 15.0...","[INVALID, 6.0, INVALID, 38.0, INVALID, 38.0, I...",0.428571,0.428571,0.428571,0.428571
8788,There are some oranges in a basket. Ana spends...,5.0,test,0.0,0.6,0.0,0.9,0.6,0.0,0.0,...,0.0,1.0,1.0,"[INVALID, INVALID, 5.0, 9.0, 2.0, INVALID, INV...","[5.0, 9.0, INVALID, 5.0, 60.0, 0.0, 30.0, 1.0,...","[5.0, 5.0, INVALID, 5.0, 5.0, INVALID, 5.0, 5....",0.238095,0.238095,0.238095,0.000000
8789,Mark's car breaks down and he needs to get a n...,230.0,test,0.0,0.4,0.0,1.0,0.3,0.0,0.0,...,1.0,0.0,0.0,"[230.0, 230.0, 230.0, 950.0, 230.0, 950.0, 950...","[230.0, 470.0, 230.0, 470.0, 470.0, 470.0, 960...","[230.0, 470.0, 80.0, 230.0, 230.0, 470.0, 470....",0.428571,0.047619,0.047619,0.428571
8790,"Farmer Brown has 20 animals on his farm, all e...",5.0,test,0.0,0.1,0.0,0.3,0.0,0.1,0.2,...,0.0,0.0,1.0,"[0.0, INVALID, 3.0, INVALID, INVALID, 10.0, 40...","[30.0, 33.0, 23.0, 15.0, 35.0, 35.0, 30.0, 20....","[15.0, 10.0, 14.0, 5.0, 5.0, 5.0, 5.0, 5.0, 17...",0.428571,0.428571,0.428571,0.428571


In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

bert_model = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model)
model = AutoModel.from_pretrained(bert_model)

def encode(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

embeddings = encode(qset)

print(embeddings.shape)

num_clusters = 50  # Number of clusters
km = KMeans(n_clusters=num_clusters, verbose=True, max_iter=1000, random_state=10)
km.fit(embeddings)
clusters = km.labels_.tolist()

df["cluster"] = clusters

In [29]:
def get_majority_vote(lst):
    return Counter(lst).most_common(n=1)[0][0]

from itertools import chain, combinations
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def best_llm_set_majority_vote_for_cluster(subdf):
    best_set, best_score = None, -1
    for llm_set in powerset(llms):
        if len(llm_set)==0:
            continue
        score = []
        for row, g in zip(subdf[[f"auto_answers_{llm}" for llm in llm_set]].to_numpy(), subdf["gold_answer"]):
            score.append(get_majority_vote([v for l in row for v in l]) == g)
        score = np.mean(score)
        # print(llm_set, score)
        if score > best_score:
            best_score = score
            best_set = llm_set
    # print(best_set)
    return best_set

llm2idx = {llm:idx for idx, llm in enumerate(llms)}
idx2llm = {idx:llm for idx, llm in enumerate(llms)}

In [30]:
# MODE = "sr"
MODE = "maj"
TH = 0.99

viable_lidxs_by_train = {}
train_df, test_df = df[df["split"]!="test"], df[df["split"]=="test"]
for c in sorted(df["cluster"].unique()):
    subdf = train_df[train_df["cluster"]==c]
    srs = [subdf[f"{MODE}_{llm}"].mean() for llm in llms]
    viable_lidxs_by_train[c] = [lidx for lidx, sr in enumerate(srs) if sr>=TH]
    # viable_lidxs_by_train[c] = [llm2idx[llm] for llm in best_llm_set_majority_vote_for_cluster(subdf)]
    if len(viable_lidxs_by_train[c]) == 0:
        viable_lidxs_by_train[c] = [np.argmax(srs)]
    print(c,len(subdf), srs, viable_lidxs_by_train[c])

0 131 [0.2366412213740458, 0.7022900763358778, 0.6717557251908397, 0.9389312977099237, 0.32061068702290074, 0.5572519083969466, 0.40458015267175573] [3]
1 161 [0.15527950310559005, 0.5714285714285714, 0.515527950310559, 0.7142857142857143, 0.33540372670807456, 0.484472049689441, 0.4658385093167702] [3]
2 169 [0.27218934911242604, 0.7988165680473372, 0.5798816568047337, 0.9467455621301775, 0.33727810650887574, 0.4911242603550296, 0.5443786982248521] [3]
3 190 [0.19473684210526315, 0.7473684210526316, 0.5473684210526316, 0.9473684210526315, 0.43157894736842106, 0.49473684210526314, 0.6421052631578947] [3]
4 216 [0.2638888888888889, 0.7268518518518519, 0.6157407407407407, 0.9490740740740741, 0.375, 0.5138888888888888, 0.49074074074074076] [3]
5 103 [0.2912621359223301, 0.7669902912621359, 0.5825242718446602, 0.9611650485436893, 0.3786407766990291, 0.6310679611650486, 0.6699029126213593] [3]
6 160 [0.23125, 0.775, 0.6125, 0.95, 0.275, 0.5, 0.50625] [3]
7 96 [0.25, 0.8020833333333334, 0.645

In [31]:
matches, total = 0, 0
viable_lidxs_by_test = {}
for c in sorted(df["cluster"].unique()):
    subdf = test_df[test_df["cluster"]==c]
    srs = [subdf[f"{MODE}_{llm}"].mean() for llm in llms]
    # viable_lidxs_by_test[c] = [llm2idx[llm] for llm in best_llm_set_majority_vote_for_cluster(subdf)]
    viable_lidxs_by_test[c] = [lidx for lidx, sr in enumerate(srs) if sr>=TH]
    if len(viable_lidxs_by_test[c]) == 0:
        viable_lidxs_by_test[c] = [np.argmax(srs)]
    print(c,len(subdf), srs, (viable_lidxs_by_train[c], viable_lidxs_by_test[c]))
    if viable_lidxs_by_train[c] == viable_lidxs_by_test[c]:
        matches += 1
        # matches += len(subdf)
    total += 1
    # total += len(subdf)
print(matches, total)

0 23 [0.17391304347826086, 0.5652173913043478, 0.5217391304347826, 0.5652173913043478, 0.17391304347826086, 0.391304347826087, 0.5217391304347826] ([3], [1])
1 37 [0.2702702702702703, 0.8378378378378378, 0.7297297297297297, 0.6756756756756757, 0.5945945945945946, 0.6216216216216216, 0.6216216216216216] ([3], [1])
2 24 [0.20833333333333334, 0.7083333333333334, 0.75, 0.75, 0.3333333333333333, 0.4166666666666667, 0.625] ([3], [2])
3 22 [0.22727272727272727, 0.6818181818181818, 0.5454545454545454, 0.8181818181818182, 0.22727272727272727, 0.36363636363636365, 0.45454545454545453] ([3], [3])
4 35 [0.17142857142857143, 0.7142857142857143, 0.5142857142857142, 0.6, 0.42857142857142855, 0.42857142857142855, 0.42857142857142855] ([3], [1])
5 19 [0.21052631578947367, 0.8421052631578947, 0.6842105263157895, 0.7894736842105263, 0.42105263157894735, 0.7894736842105263, 0.5789473684210527] ([3], [1])
6 23 [0.08695652173913043, 0.782608695652174, 0.6956521739130435, 0.6086956521739131, 0.30434782608695

In [32]:
def get_majority_vote(lst):
    return Counter(lst).most_common(n=1)[0][0]

print("Upper Bound Q-Wise\t\t\t",test_df[[f"{MODE}_{llm}" for llm in llms]].max(axis=1).mean(),f"\tUsage: {1/ len(llms)}")

test_auto_answers = test_df[[f"auto_answers_{llm}".removesuffix("_CHAT") for llm in llms]].to_numpy()
maj_list = []
usage = []
for taa, c, g in zip(test_auto_answers, test_df['cluster'], test_df["gold_answer"]):
    auto_ans_list = []
    u = 0
    for lidx, a in enumerate(taa):
        if lidx in viable_lidxs_by_test[c]:
            auto_ans_list.extend(a)
            u += 1
    usage.append(u)
    maj_list.append(get_majority_vote(auto_ans_list) == g)
mean_maj = np.mean(maj_list)
usage = np.mean(usage) / len(llms)
print("Upper Bound Cluster-Wise\t\t",mean_maj,f"\tUsage: {usage}")

maj_list = []
usage = []
_MODE = "maj"
for taa, c, g in zip(test_auto_answers, test_df['cluster'], test_df["gold_answer"]):
    auto_ans_list = []
    u = 0
    for lidx, a in enumerate(taa):
        # if lidx in viable_lidxs_by_test[c]:
        if lidx in viable_lidxs_by_train[c]:
            auto_ans_list.extend(a)
            u += 1
    usage.append(u)
    maj_list.append(get_majority_vote(auto_ans_list) == g)
mean_maj = np.mean(maj_list)
usage = np.mean(usage) / len(llms)
print("Predicted:  Cluster-Wise\t\t",mean_maj,f"\tUsage: {usage}")

# for lidx, llm in enumerate(llms):
#     print(f"Predicted:  Always {llm}\t",np.mean([row[lidx] for row in test_df[[f"{_MODE}_{llm}" for llm in llms]].to_numpy()]),f"\tUsage: {1/len(llms)}")

from itertools import chain, combinations
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

for lidx_set in powerset(range(len(llms))):
    if len(lidx_set)==0:
        continue
    maj_list = []
    usage = []
    for taa, c, g in zip(test_auto_answers, test_df['cluster'], test_df["gold_answer"]):
        auto_ans_list = []
        u = 0
        for lidx, a in enumerate(taa):
            if lidx in lidx_set:
                auto_ans_list.extend(a)
                u += 1
        usage.append(u)
        maj_list.append(get_majority_vote(auto_ans_list) == g)
    mean_maj = np.mean(maj_list)
    usage = np.mean(usage) / len(llms)
    print(f"Predicted: {[llms[lidx] for lidx in lidx_set]}\t\t",mean_maj,f"\tUsage: {usage}")

Upper Bound Q-Wise			 0.8809704321455648 	Usage: 0.14285714285714285
Upper Bound Cluster-Wise		 0.7354056103108415 	Usage: 0.14285714285714285
Predicted:  Cluster-Wise		 0.6724791508718726 	Usage: 0.14285714285714285
Predicted: ['llama2-7b-lm']		 0.22365428354814254 	Usage: 0.14285714285714285
Predicted: ['gemma-7b']		 0.7126611068991661 	Usage: 0.14285714285714285
Predicted: ['mistral-7b-lm']		 0.6050037907505686 	Usage: 0.14285714285714285
Predicted: ['metamath-7b']		 0.6724791508718726 	Usage: 0.14285714285714285
Predicted: ['gemma-7b-it']		 0.39727065959059893 	Usage: 0.14285714285714285
Predicted: ['llama2-13b-chat']		 0.49052312357846856 	Usage: 0.14285714285714285
Predicted: ['mistral-7b-inst']		 0.5360121304018196 	Usage: 0.14285714285714285
Predicted: ['llama2-7b-lm', 'gemma-7b']		 0.645185746777862 	Usage: 0.2857142857142857
Predicted: ['llama2-7b-lm', 'mistral-7b-lm']		 0.5109931766489765 	Usage: 0.2857142857142857
Predicted: ['llama2-7b-lm', 'metamath-7b']		 0.6724791508718

### ITER

In [ ]:
from itertools import chain, combinations
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# MODE = "sr"
MODE = "maj"
# TH = 0.5

plotdf = []

for TH in np.linspace(0,1,50):

    viable_lidxs_by_train = {}
    train_df, test_df = df[df["split"]!="test"], df[df["split"]=="test"]
    for c in sorted(df["cluster"].unique()):
        subdf = train_df[train_df["cluster"]==c]
        srs = [subdf[f"{MODE}_{llm}"].mean() for llm in llms]
        viable_lidxs_by_train[c] = [lidx for lidx, sr in enumerate(srs) if sr>=TH]
        if len(viable_lidxs_by_train[c]) == 0:
            viable_lidxs_by_train[c] = [np.argmax(srs)]
        # print(c,len(subdf), srs, viable_lidxs_by_train[c])

    matches, total = 0, 0
    viable_lidxs_by_test = {}
    for c in sorted(df["cluster"].unique()):
        subdf = test_df[test_df["cluster"]==c]
        srs = [subdf[f"{MODE}_{llm}"].mean() for llm in llms]
        viable_lidxs_by_test[c] = [lidx for lidx, sr in enumerate(srs) if sr>=TH]
        if len(viable_lidxs_by_test[c]) == 0:
            viable_lidxs_by_test[c] = [np.argmax(srs)]
        # print(c,len(subdf), srs, (viable_lidxs_by_train[c], viable_lidxs_by_test[c]))
            
    def get_majority_vote(lst):
        return Counter(lst).most_common(n=1)[0][0]

    test_auto_answers = test_df[[f"auto_answers_{llm}".removesuffix("_CHAT") for llm in llms]].to_numpy()

    maj_list = []
    usage = []
    for taa, c, g in zip(test_auto_answers, test_df['cluster'], test_df["gold_answer"]):
        auto_ans_list = []
        u = 0
        for lidx, a in enumerate(taa):
            if lidx in viable_lidxs_by_train[c]:
                auto_ans_list.extend(a)
                u += 1
        usage.append(u)
        maj_list.append(get_majority_vote(auto_ans_list) == g)
    mean_maj = np.mean(maj_list)
    usage = np.mean(usage) / len(llms)
    
    print(TH,mean_maj,usage)
    plotdf.append({
        "threshold": TH,
        "mean_maj": mean_maj,
        "usage": usage,
    })

0.0 0.7437452615617892 1.0
0.02040816326530612 0.7437452615617892 1.0
0.04081632653061224 0.7437452615617892 1.0
0.061224489795918366 0.7437452615617892 1.0
0.08163265306122448 0.7437452615617892 1.0
0.1020408163265306 0.7437452615617892 1.0
0.12244897959183673 0.7437452615617892 1.0
0.14285714285714285 0.7437452615617892 0.9967507852269035
0.16326530612244897 0.7445034116755117 0.9927434203400844
0.18367346938775508 0.7452615617892343 0.9772554965883246
0.2040816326530612 0.7475360121304018 0.9542943788584425
0.22448979591836732 0.7482941622441244 0.9330661756742121
0.24489795918367346 0.7505686125852918 0.9168201018087296
0.26530612244897955 0.7536012130401819 0.8936423697606413
0.2857142857142857 0.7543593631539045 0.8703563305534496
0.3061224489795918 0.7581501137225171 0.8458789125961227
0.32653061224489793 0.7604245640636846 0.8232427163435503
0.3469387755102041 0.7611827141774071 0.7898841113397596
0.36734693877551017 0.7619408642911296 0.7763457164518576
0.3877551020408163 0.76

In [ ]:
plotdf = pd.DataFrame(plotdf)
fig = px.line(plotdf, x="threshold", y="mean_maj")
fig.show()
fig = px.line(plotdf, x="threshold", y="usage")
fig.show()